This is program is to extract opensmile features, which are used for comparison with speech attribute scores and fbank_pitch features. There are two approachs:
- Approach-1:​

- - Compute 25-dim LLD features of eGeMAPS and corresponding 1st and 2nd order of dynamic features for each frame (width=25ms, shift=10ms)​

- - Each 1 s segment is represented by a 100X75 feature matrix​


In [ ]:
import opensmile
import audiofile
import os
import numpy as np
import pandas as pd
from glob import glob

In [ ]:
# parameters definition
# task = 'PC-GITA_vowel'
# task = 'PC-GITA_read'
task = 'PC-GITA_spon'
# task = 'PDSTU_vowel'
# task = 'PDSTU_read'
# task = 'PDSTU_spon'
filepath_audio = os.path.join('/home/yuanyuan/Documents/MULAN-ACCENT/data/', task)
filepath_res = os.path.join('/home/yuanyuan/Documents/MULAN-ACCENT/opensmile/', task)
if os.path.exists(filepath_res) == False:
    os.makedirs(filepath_res)
segment_length = 1 # 100 frames, 1 s
segment_shift =  0.2 # frames, 0.2 s
frame_length = 0.025 # 25 ms
frame_shift = 0.010 # 10 ms

smile_lld = opensmile.Smile(
    feature_set = opensmile.FeatureSet.eGeMAPSv02,
    feature_level = opensmile.FeatureLevel.LowLevelDescriptors,
)    
smile_lld.feature_names, len(smile_lld.feature_names)

In [ ]:
import opensmile
print(opensmile.__version__)
print(opensmile.__file__)

In [ ]:
# to compute frame-level lld features and functionals for each 1 s segments for given audio file.
def feature_extraction(file, smile_lld, smile_func, frame_shift, segment_length, segment_shift, filepath_res):
    audio_name = os.path.basename(file)
    audio_name = audio_name[0:audio_name.index('.wav')]
    if task in audio_name == False:
        audio_name = audio_name + '_' + task
    audio, sampling_rate = audiofile.read(file)
    delete_flag = 0
    if audio.shape[0] >= sampling_rate:    
        df_feats_lld = smile_lld.process_signal(audio, sampling_rate)
        start_lld = list(df_feats_lld.index)
        if start_lld[-1][0] == start_lld[-2][0]:
            cut_lastrow = 1
            df_feats_lld = df_feats_lld[:-1]
#             print('cut the last row: lld')
        else:
            cut_lastrow = 0
        frameid_lld = [int(np.around(start_lld[i][0].total_seconds()/frame_shift)) for i in range(len(start_lld)-cut_lastrow)]
    #     if frameid_lld[-1] == frameid_lld[-2]:
    #         frameid_lld = frameid_lld[0:-1]
    #         
        if len(frameid_lld) != len(np.unique(frameid_lld)):
            print('lld index generation error for {}'.format(audio_name))
        df_feats_lld.index = frameid_lld
        df_feats_lld.to_excel(os.path.join(filepath_res, audio_name+'_lld.xlsx'))
    else:
        print('delete {}, shorter than 1 s.'.format(audio_name))
        delete_flag = 1
    return delete_flag

In [ ]:
# run feature_extraction for a bunch of audios of given task.
files = glob(os.path.join(filepath_audio, '*.wav'))
delete_wavs = 0
print(filepath_res)
for file in files:
    delete_flag = feature_extraction(file, smile_lld, smile_func, frame_shift, segment_length, segment_shift, filepath_res)
    delete_wavs = delete_wavs + delete_flag
print('original {} wavs, deleted {} wavs'.format(len(files), delete_wavs))